### 1. 概述
#### 1.1 Numpy和Pandas的区别
||Numpy|Pandas|
|:----|:----|:----|
|作用|数值计算|数据处理|
|主要数据结构|ndarray|Series,DataFrame|
|数据类型|ndarry同构但可以向上取(eg:np.array([1, '1', "1"]))|Series同构但可以向上取(eg:pd.Series([1, '1', "1"]))|


### 2. 基本数据结构
#### 2.1 Series
+ 带标签(index)的一维数组
+ 创建方式：pd.Series(data, index=index) (index形式灵活，基本都可以识别出来)
    + data不可为set,但index可为set,此时index无序，随机
+ data支持的数据类型:
    + Python 字典
    + 多维数组(不能是set，set无序)
        + 未设置index时：
           + Python 版本 >= 3.6 且 Pandas 版本 >= 0.23 Series按字典的插入顺序排序索引
           + Python < 3.6 或 Pandas < 0.23 Series按字母顺序排序字典的键（key）列表
        + 设置index时：
           + 按索引标签提取data里对应的值
    + 标量值（如1）
        + data为标量值时，必须提供index，Series按index长度重复
+ 基本操作：
    + 与ndarray类似，支持大多数Numpy函数，同时支持索引切片，且可以与numpy进行转换
    + 与字典类似，可以用index索引提取值或设置值
        + a['1']表示index为'1' 的，a[1] 表示第二个数
        + 上面这种冲突可以用a.get(1) 解决
    + 矢量操作与对齐Series标签
        + Series与Numpy数组类似，支持大多数类似方法
        + Series之间的操作会自动基于标签对齐，没有的标记为NaN
    + Series支持自定义name属性

In [17]:
import pandas as pd
import numpy as np
pd.Series(np.array(range(5)), (1,2,3,4,5)) # index 形式灵活，基本都可以识别出来

### data 为多维数组
pd.Series((1,2,3))

### data 为字典
pd.Series({'b': 1, 'a': 0, 'c': 2})
pd.Series({'b': 1, 'a': 0, 'c': 2}, index=['1', 'b', 'c', 'd']) #按索引标签提取

### data为标量值
pd.Series(1, index=['a', 'b'])  #必须提供index，按index长度重复该标量值

a    1
b    1
dtype: int64

In [48]:
### 与numpy操作类似
a = pd.Series(np.array([3,2,1,0]))
a.dtype
a[0]    #第一个值
a[[0]]  #index为0的值
a[:3]   #支持切片
a[a > np.min(a)] #支持一些np常规操作
a.dtype  #支持dtype
a.array  #Series.arry 拓展数组
type(a.to_numpy()) #可转成numpy的ndarray

numpy.ndarray

In [67]:
### 与dic类似
a['0'] = "111"   #a['1'] 和 a[1] 不是同一个数
a = pd.Series(np.array([1,2,3]))
a, a.get('1')

(0    1
 1    2
 2    3
 dtype: int64, None)

In [80]:
b = pd.Series(np.array([1,2,3,4]), index=[3,2,1,0])
a + b  #自动根据标签相加,没有的标记为NaN

0    5.0
1    5.0
2    5.0
3    NaN
dtype: float64

In [81]:
a.rename('dif')

0    1
1    2
2    3
Name: dif, dtype: int64

#### 2.2 DataFrame
+ DataFrame 是由多类型的列构成的二维标签数据结构，类似SQL表
+ 支持多种类型的数据输入：
    + 一维ndarry，列表，字典，Series字典
    + 二维numpy.ndarry
    + Series，DataFrame
    + 除了数据，还可以有选择的传递index（行标签）和columus（列标签）参数
+ 创建方式：
    + 用Series字典或字典创建
        + 指定列与数据字典一起传递时，传递的列会覆盖字典的键
    + 用多维数组字典、列表字典创建
        + 多维数组长度必须相同。
            + 传递index：index长度必须与数组长度相同
            + 未传递index：index为range(n)，n为数组长度
    + 用列表字典生成DataFrame
    + 用元组字典生成DataFrame
        + 和列表差不多
    + 用Series创建DataFrame
+ 基本操作：
    + 类似字典操作 列
    + 获取某一行
        + df.loc[label],df.iloc[loc]

In [91]:
# 用Series字典或字典创建
a = {'one': pd.Series(np.array([1,2,3]), index=['a','b','c']),
     'two': pd.Series(np.array([1,2,3,4]), index=['a','b','c','d'])}
df = pd.DataFrame(a)
df1 = pd.DataFrame(a, columns=['two', 'three'])
print(df, df1)
pd.DataFrame(df, index=['d','b','a']) #index 用于访问行
pd.DataFrame(df, index=['d','b','a'], columns=['two','three']) #columns用于访问列

   one  two
a  1.0    1
b  2.0    2
c  3.0    3
d  NaN    4    two three
a    1   NaN
b    2   NaN
c    3   NaN
d    4   NaN


,two,three
d,4,NaN
b,2,NaN
a,1,NaN


In [98]:
# 用多维数组字典、列表字典创建
df = pd.DataFrame({'one': [1,2,3,4],
                   'two': [4,3,2,1]}) #index 未指定，默认range(n)
# df = pd.DataFrame({'one': [1,2,3,4],
#                    'two': [4,3,2]})   #arrays must all be same length
df = pd.DataFrame({'one': [1,2,3,4],
                   'two': [4,3,2,1]}, index = ['a', 'b', 'c', 'd'])
df

,one,two
a,1,4
b,2,3
c,3,2
d,4,1


In [100]:
# 用列表字典生成DataFrame
data = [{'a': 1, 'b': 2},{'a': 3, 'c': 4, 'd': 5},{'a': 6, 'e':8}]
pd.DataFrame(data) #多个dict合并

,a,b,c,d,e
0,1,2.0,NaN,NaN,NaN
1,3,NaN,4.0,5.0,NaN
2,6,NaN,NaN,NaN,8.0


In [112]:
# 用元组字典生成DataFrame
data = ({'a': 1, 'b': 2},{'a': 3, 'c': 4, 'd': 5},{'a': 6, 'e':8})
df = pd.DataFrame(data, index=['1', '2', '3']) #多个dict合并
df

,a,b,c,d,e
1,1,2.0,NaN,NaN,NaN
2,3,NaN,4.0,5.0,NaN
3,6,NaN,NaN,NaN,8.0


In [119]:
# 类似字典操作列
df = pd.DataFrame(data, index=['1', '2', '3']) #多个dict合并
df['a']
del df['a'] #删除某一列
df
e = df.pop('e')
e, type(e) #弹出某一列为 Series
#df['1'] 只会查找列元素
df['f'] = '123' #以广播方式填充
df

,b,c,d,f
1,2.0,NaN,NaN,123
2,NaN,4.0,5.0,123
3,NaN,NaN,NaN,123
